# I. Introduction to `premise`

`premise` requires an ecoinvent database to build on.
This database can either be from a `brihgtway` project, or as *ecoSpold2* files from the [ecoinvent](htpps://ecoinvent.org) website.

## I.1 Fetching ecoinvent

If you do not have a `brightway` project with ecoinvent, here is how to get it.

In [ ]:
import eidl
import bw2data, bw2io

In [ ]:
# create a brightway project
bw2data.projects.set_current('brightcon23')

In [ ]:
# set up the project with biosphere database and LCIA methods
bw2io.bw2setup()

In [ ]:
bw2data.databases

In [ ]:
# fetch ecoinvent
eidl.get_ecoinvent()

Or manually...

In [ ]:
fp="/Users/romain/Downloads/ecoinvent 3.8_cutoff_ecoSpold02/datasets"
ei=bw2io.importers.SingleOutputEcospold2Importer(fp, "ecoinvent 3.8 cutoff")
ei.apply_strategies()
ei.match_database(fields=["name", "reference product", "location", "unit"])
ei.match_database("biosphere3", fields=["name", "categories", "unit"])

In [ ]:
ei.statistics()

In [ ]:
ei.write_database()

## I.2 The premise bit

In [ ]:
from premise import *

### I.2.1 With ecoinvent in a brightway project

In [ ]:
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
    ],
    source_db="ecoinvent 3.8 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.8", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=' # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
)

### I.2.2 Or from ecoSpold2 files

In [ ]:
ndb = NewDatabase(
        scenarios=[
            {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
            {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
        ],      
        source_type="ecospold", # <--- this is NEW
        source_file_path="/Users/romain/Downloads/ecoinvent 3.8_cutoff_ecoSpold02/datasets", # <-- this is NEW
        source_version="3.8",
)


### I.3 Printing out scenario data

In [ ]:
ndb.generate_scenario_report()

### I.4 Exploring the database

In [ ]:
# print the first activity
ndb.scenarios[0]["database"][0]

In [ ]:
# printing all non-ecoinvent datasets
ds = (d for d in ndb.scenarios[0]["database"] if d.get("database") != "ecoinvent 3.8 cutoff")
for d in ds:
    print(d["name"])

### I.5 updating the electricity sector

ndb.update_electricity()

### I.6 Or everything!

In [ ]:
ndb.update_all()

# II. Export

## II.1 Export back to a `brightway` project

In [ ]:
ndb.write_db_to_brightway("my prospective db")

## II.2 Export to Simapro

ndb.write_db_to_simapro()

## II.3 Superstructure databases and `activity-browser`

In [ ]:
ndb = NewDatabase(
        scenarios=[
            {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2020},
            {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
            {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2080},
        ],      
        source_type="ecospold", # <--- this is NEW
        source_file_path="/Users/romain/Downloads/ecoinvent 3.8_cutoff_ecoSpold02/datasets", # <-- this is NEW
        source_version="3.8",
)
ndb.update_electricity()
ndb.write_superstructure_db_to_brightway()

## II.4 Or share the databases as datapackage

In [ ]:
ndb.write_datapackage()

# III. Unfold datapackages

## III.1 Unfold from an online or local repository

In [ ]:
from unfold import Unfold
fp = "/Users/romain/Downloads/premise_remind_ssp2_base.zip"
u = Unfold(fp)